In [1]:
import sqlalchemy
import db_module

create_new_test_db = True
import os
if os.path.exists("tests.sqlite"): #Если файл уже есть, заморачиваться с перезаполнением не будем
    create_new_test_db = False

## Создаем тестовую БД методом класса dbModule

In [2]:
myDB = db_module.dbModule(dbstring = 'sqlite:///tests.sqlite')
if create_new_test_db:
    myDB.create_sqlite_file()

## Напечатаем названия таблиц в БД для проверки

In [3]:
db_module.Base.metadata.tables.keys()

dict_keys(['image', 'dataset', 'category', 'license', 'annotation', 'trainResult', 'categoryToModel', 'model'])

## Заполним базу аннотациями из COCO -- это необходимо делать один раз

In [ ]:
dataDir=r"D:\Work\AutoML\annotations_trainval2017"
dataType='train2017'
annFile=r"{}\annotations\instances_{}.json".format(dataDir,dataType)


import time

begin = time.time()
if create_new_test_db:
    myDB.fill_cats_dogs(file_prefix = './datasets/Kaggle/') #Загружаем базу Kaggle - кошки и собаки
    myDB.fill_coco(annFile, file_prefix = './datasets/COCO2017/', firstTime = True) #Надо указать True чтобы подгрузить категории 
end = time.time()

print('DB filled in', end-begin, 'seconds')

Start filling DB with Kaggle CatsVsDogs
Finished with Kaggle CatsVsDogs
loading annotations into memory...
Done (t=41.21s)
creating index...
index created!
Adding 860001 annotations in COCO format to DB
Adding images
Done adding images, adding annotations


## Добавляем информацию о моделях в БД - эта информация может в дальнейшем использоваться для рекомендаций при подготовке новых моделей

In [ ]:
myDB.add_model_record(task_type = "detection", categories = ['cat','mouse'], model_address = './TrainedNN/New_NetworkFirst', metrics = {'accuracy':0.91}, history_address = '')
myDB.add_model_record(task_type = "classification", categories = ['cat','dog'], model_address = './TrainedNN/New_NetworkPP', metrics = {'accuracy':0.91}, history_address = '')
myDB.add_model_record(task_type = "classification", categories = ['cat','dog'], model_address = './TrainedNN/New_NetworkFirst2', metrics = {'accuracy':0.80}, history_address = '')
myDB.add_model_record(task_type = "classification", categories = ['cat','mouse'], model_address = './TrainedNN/New_NetworkPP2', metrics = {'accuracy':0.93}, history_address = '')
myDB.add_model_record(task_type = "classification", categories = ['cat','mouse'], model_address = './TrainedNN/New_NetworkFirst3', metrics = {'loss':0.93}, history_address = '')

## Пример запроса к базе - ищем модель, удовлетворяющую критериям: "минимальная точность - 0.92, категории, с которыми работает модель - кошки, мыши". Поиск идет на включение категорий, без точного совпадения

In [ ]:
myDB.get_models_by_filter({'min_metrics':{'accuracy':0.91}, 'categories':['cat','mouse']})

# Взаимодействие с добавленными результатами обучения - попытка дубля, модификация, удаление 

In [ ]:
#При попытке добавить информацию о существующей модели второй раз пользователь получает ошибку
#Модель однозначно идентифицируется полем model_address. Изменить категории для данной модели и тип задачи - пока нельзя (может быть в будущем подход будет пересмотрен)
myDB.add_model_record(task_type = "classification", categories = ['cat','mouse'], model_address = './TrainedNN/New_NetworkFirst3', metrics = {'loss':0.94}, history_address = '')

In [ ]:
#Однако поправить результат обучения вполне можно, также как и добавить новую метрику
myDB.update_train_result_record(model_address = './TrainedNN/New_NetworkPP2', metric_name = 'accuracy', metric_value = 0.97, history_address = '')

In [ ]:
#Проверяем результат обновления
myDB.get_models_by_filter({'min_metrics':{'accuracy':0.1}, 'categories':['cat','mouse']})

In [ ]:
#Можно удалить результат обучения
myDB.delete_train_result_record(model_address = './TrainedNN/New_NetworkPP2', metric_name = 'accuracy')

In [ ]:
#Проверяем результат удаления
myDB.get_models_by_filter({'min_metrics':{'accuracy':0.9}, 'categories':['cat','mouse']})

In [ ]:
#Выводим список вообще всех моделей:
myDB.get_models_by_filter({})

In [ ]:
#Можно запросить модели с фильтрами по ID категорий, а не по названиям:
myDB.get_models_by_filter({'categories_ids':[17,18]})

## Пример запроса к базе - получаем перечень всех доступных категорий

In [ ]:
myDB.get_all_categories()

## Пример запроса к базе - получаем аннотации для двух категорий - кошки и собаки

In [ ]:
myDB.load_specific_categories_annotations(['cat','dog'], normalizeCats = True, splitPoints = [0.7, 0.85], curExperimentFolder = './', crop_bbox = False, cropped_dir = './crops')

## Пример запроса к базе - получаем список всех подгруженных датасетов

In [ ]:
myDB.get_all_datasets()

## Пример запроса к базе - подгружаем аннотации из определенных датасетов (по ID)

In [ ]:
myDB.load_specific_datasets_annotations([1]) #ID 1 как видно выше - это база Kaggle

# Пример запроса к базе - получить ID категорий по названию и наоборот

In [ ]:
cat_IDs = myDB.get_cat_IDs_by_names(["cat","dog","fish","house","bird","person"])
print("cat_IDs:",cat_IDs)
cat_names = myDB.get_cat_names_by_IDs(cat_IDs)
print("cat_names:",cat_names)